In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from IPython.display import Audio

import requests

url = "https://raw.githubusercontent.com/k5yi/econ2005/master/datasets/kid_laugh.wav"
laugh = requests.get(url).content

with open('kid_laugh.wav', 'wb') as f:
    f.write(laugh)

fs, data = read('kid_laugh.wav')

In [12]:
def reverse_play(sample):
    step = -1
    return int(fs), sample[::step, :]

In [13]:
fs, rev_sound = reverse_play(data)
Audio(rev_sound.T, rate=fs)

In [14]:
import math
def amplify(sample, level = 6):
    rlevel = math.pow(10,(level / 20))
    return int(fs), sample * rlevel

In [15]:
fs, amplified = amplify(data, level=6)
Audio(amplified.T, rate=fs)

In [16]:
def echo_effect(sample, fs = 44100, delay = 0.5, repeat = 2, effect = .2):
    arr = np.zeros((sample.shape[0] * 2, 2))
    for x in range(sample.shape[0]):
        arr[x][0] = sample[x][0]
        arr[x][1] = sample[x][1]
    ind = sample.shape[0] * delay
    ind = int(ind)
    tmp = sample
    for y in range(0, repeat, 1):
        tmp = tmp * (1 - effect)
        ind = ind * (y + 1)
        for z in range(sample.shape[0]):
            arr[z + ind][0] += tmp[z][0]
            arr[z + ind][1] += tmp[z][1]
    return arr

In [17]:
echo = echo_effect(data, fs=44100, delay=0.5, repeat=2, effect=.2)
Audio(echo.T, rate=fs)

In [18]:
import requests

url = "https://raw.githubusercontent.com/k5yi/econ2005/master/datasets/woosh.wav"
woosh = requests.get(url).content

with open('woosh.wav', 'wb') as f:
    f.write(woosh)

fs_woosh, data_woosh = read('woosh.wav')

In [19]:
def mixing(sample_list, weight = (.4, .6)):
    if sample_list[0].shape[0] >= sample_list[1].shape[0]:
        return_arr = sample_list[0] * weight[0]
        tmp_arr = sample_list[1] * weight[1]
        tmp_arr = np.resize(tmp_arr, (return_arr.shape[0], return_arr.shape[1]))
    else:
        return_arr = sample_list[1] * weight[1]
        tmp_arr = sample_list[0] * weight[0]
    tmp_arr = np.resize(tmp_arr, (return_arr.shape[0], return_arr.shape[1]))
    return_arr = return_arr + tmp_arr
    return int(fs), return_arr

In [20]:
fs_mix, data_mix = mixing([data, data_woosh], weight = (.4, .6))
Audio(data_mix.T, rate=fs_mix)